In [1]:
!conda install -c conda-forge pyarrow -y
!pip install --user --upgrade pyarrow

!pip3 uninstall fsspec -y
!pip install fsspec -U

!pip install git-lfs

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pyarrow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    arrow-cpp-8.0.0            |   py37h60b952e_0         9.5 MB
    aws-c-common-0.4.57        |       he6710b0_1         156 KB
    aws-c-event-stream-0.1.6   |       h2531618_5          25 KB
    aws-checksums-0.1.9        |       he6710b0_0          49 KB
    aws-sdk-cpp-1.8.185        |       hce553d0_0         1.9 MB
    boost-cpp-1.80.0           |       h75c5d50_0        17.4 MB  conda-forge
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB

In [2]:
!pip install -r /kaggle/input/russian-lyrics/lyrics_generation_api/lyrics_generation_api/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 8.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 28.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [3]:
import glob
import os
import re
from typing import List

import evaluate
import pandas as pd
import torch
import transformers
from datasets import Dataset, load_dataset
from tqdm import tqdm
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          default_data_collator)

In [4]:
import sys

sys.path.insert(1, '/kaggle/input/russian-lyrics/lyrics_generation_api/')

from lyrics_generation_api.src.models.gpt_utils import preprocess_logits_for_metrics, generate_song
from lyrics_generation_api.src.data.get_dataset import get_custom_tokenize

# Training

In [5]:
metric = evaluate.load("accuracy")


def compute_metrics(eval_preds, metric):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics but we need to shift the labels
    labels = labels[:, 1:].reshape(-1)
    preds = preds[:, :-1].reshape(-1)
    return metric.compute(predictions=preds, references=labels)

In [5]:
# tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
# model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

# special_tokens_dict = {
#         "additional_special_tokens": [
#             '[s:artist]', '[s:song_name]', '[s:lyrics]',
#             '[e:artist]', '[e:song_name]', '[e:lyrics]'
#             ],
#         'pad_token': '[PAD]'
#        }

# tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer))

# # dataset = load_dataset('Milana/russian-indi-alternative')
# dataset = dataset.map(lambda examples: get_custom_tokenize(examples, tokenizer), desc='Tokenizing data')
# dataset = dataset.remove_columns(['artist', 'song_name', 'lyrics', 'Unnamed: 0'])
# dataset.set_format('pt', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'], output_all_columns=True)

# os.mkdir('/kaggle/working/training')

In [ ]:
train_args = transformers.TrainingArguments(
                warmup_steps=3,
                load_best_model_at_end=True,
                num_train_epochs=1,
                per_device_train_batch_size=8,
                save_strategy = "no",
                output_dir='/kaggle/working/training'
)

trainer = transformers.Trainer(
            args=train_args,
            model=model,
            train_dataset=dataset['train'],
            compute_metrics=compute_metrics,
            preprocess_logits_for_metrics=preprocess_logits_for_metrics,
            data_collator=default_data_collator,
)

# to train again run  the next line
# trainer.train()

# Generate

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Milana/russian_alternative_indi")
model = AutoModelForCausalLM.from_pretrained("Milana/russian_alternative_indi")

Downloading:   0%|          | 0.00/753 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/863 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
generated_song = generate_song('Noize MC', 'новый год', '', model, tokenizer, temperature=0.9, max_length=256, top_k=80)
print(generated_song)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
